In [1]:
import numpy as np
import pandas as pd
from keras import layers
from keras.layers import Input,Dense,Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D
from keras.layers import AveragePooling2D,MaxPooling2D,Dropout,GlobalMaxPooling2D,GlobalAveragePooling2D
from keras.utils import np_utils,print_summary
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data=pd.read_csv("data2.csv")
dataset=np.array(data)
np.random.shuffle(dataset)
#X = dataset.iloc[:,:-1]
#Y = dataset.iloc[:,-1]
#rows_to_remove = np.where(dataset.iloc[:,1024].values==1024)
#dataset = dataset.drop(dataset.index[rows_to_remove[0]])
X=dataset
Y=dataset

X=X[:,0:1024]
Y=Y[:,1024]

X_train=X[0:70000,:]
X_train=X_train/255.
X_test=X[70000:72001,:]
X_test=X_test/255.

#Reshape
Y=Y.reshape(Y.shape[0],1)
Y_train=Y[0:70000,:]
Y_train=Y_train.T
Y_test=Y[70000:72001,:]
Y_test=Y_test.T

In [3]:
print("number of training examples =" + str(X_train.shape[0]))
print("number of test examples =" + str(X_test.shape[0]))
print("X_train shape: "+ str(X_train.shape))
print("Y_train shape: "+ str(Y_train.shape))
print("X_test shape: "+ str(X_test.shape))
print("Y_test shape: "+ str(Y_test.shape))

number of training examples =70000
number of test examples =2001
X_train shape: (70000, 1024)
Y_train shape: (1, 70000)
X_test shape: (2001, 1024)
Y_test shape: (1, 2001)


In [4]:
image_x=32
image_y=32
#rows_to_remove = np.where(dataset.iloc[:,1024].values==1024)
#dataset = dataset.drop(dataset.index[rows_to_remove[0]])
train_y=np_utils.to_categorical(Y_train)
test_y=np_utils.to_categorical(Y_test)
train_y=train_y.reshape(train_y.shape[1],train_y.shape[2])
test_y=test_y.reshape(test_y.shape[1],test_y.shape[2])
X_train=X_train.reshape(X_train.shape[0],image_x,image_y,1)
X_test=X_test.reshape(X_test.shape[0],image_x,image_y,1)

In [5]:
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(train_y.shape))

X_train shape: (70000, 32, 32, 1)
Y_train shape: (70000, 48)


In [6]:
def keras_model(image_x,image_y):
    num_of_classes=48
    model=Sequential()
    model.add(Conv2D(filters=32,kernel_size=(5,5),input_shape=(image_x,image_y,1),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same'))
    model.add(Conv2D(64,(5,5),activation='relu'))
    model.add(MaxPooling2D(pool_size=(5,5),strides=(5,5),padding='same'))
    #model.add(Conv2D(128,(5,5),activation='relu'))
    #model.add(MaxPooling2D(pool_size=(5,5),strides=(5,5),padding='same'))
    model.add(Flatten())
    model.add(Dense(num_of_classes,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    filepath="devanagari.h5"
    checkpoint1=ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only=True,mode='max')
    callbacks_list=[checkpoint1]

    return model,callbacks_list

In [7]:
model,callbacks_list=keras_model(image_x,image_y)
model.fit(X_train,train_y,validation_data=(X_test,test_y),epochs=4,batch_size=64,callbacks=callbacks_list)
scores=model.evaluate(X_test,test_y,verbose=0)
print("CNN Error: %.2f%%" % (100 - scores[1] * 100))
print_summary(model)
model.save('devanagari.h5')

Train on 70000 samples, validate on 2001 samples
Epoch 1/4
70000/70000 [==============================] - 623s 9ms/step - loss: 0.8228 - acc: 0.7727 - val_loss: 0.2997 - val_acc: 0.9155

Epoch 00001: val_acc improved from -inf to 0.91554, saving model to devanagari.h5
Epoch 2/4
70000/70000 [==============================] - 616s 9ms/step - loss: 0.2350 - acc: 0.9304 - val_loss: 0.2116 - val_acc: 0.9350

Epoch 00002: val_acc improved from 0.91554 to 0.93503, saving model to devanagari.h5
Epoch 3/4
70000/70000 [==============================] - 609s 9ms/step - loss: 0.1505 - acc: 0.9545 - val_loss: 0.1366 - val_acc: 0.9565

Epoch 00003: val_acc improved from 0.93503 to 0.95652, saving model to devanagari.h5
Epoch 4/4
70000/70000 [==============================] - 612s 9ms/step - loss: 0.1114 - acc: 0.9654 - val_loss: 0.1519 - val_acc: 0.9515

Epoch 00004: val_acc did not improve from 0.95652
CNN Error: 4.85%
_________________________________________________________________
Layer (type)  